Nama : Siti Ma'rifatul Khoiriyah

Nim : 210411100007

Kelas : PPW A

# **Tugas 2 Preprocessing Dan TF IDF**

**PRE PROCESSING**



---
Preprocessing adalah proses membersihkan dan mempersiapkan data mentah agar siap digunakan oleh model machine learning. Ini meliputi penanganan data yang hilang, normalisasi, mengubah data kategori menjadi angka, dan membersihkan teks. Tujuannya agar data lebih mudah dipahami dan diolah oleh model untuk hasil yang lebih akurat, Berikut adalah beberapa langkah umum dalam pre-processing teks:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/PPW/report/tugas_ppw/databerita100.csv")
df.head()

,judul,tanggal,isi,kategori
0,Potret Cantik Enzy Storia Saat Kulineran di Pa...,"Rabu, 25 Sep 2024 22:00 WIB",Jakarta - Enzy Storia banjir pujian setelah ta...,Makanan
1,Potret Mesra Sahrul Gunawan Saat Kulineran Bar...,"Rabu, 25 Sep 2024 21:00 WIB",Jakarta - Sahrul Gunawan dan istri gagal punya...,Makanan
2,Ditegur Staf Restoran Gegara Bawa Makanan dari...,"Rabu, 25 Sep 2024 19:30 WIB",Jakarta - Seorang pengunjung restoran jadi sak...,Makanan
3,Unik! Burger Ini Isinya Kepiting Utuh Berukura...,"Rabu, 25 Sep 2024 19:00 WIB",Jakarta - Biasanya burger berisi patty atau da...,Makanan
4,"Qobe Mie: Sensasi Mie Pedas Baru di Bogor, Lar...","Rabu, 25 Sep 2024 18:41 WIB",Jakarta - Ingin coba menikmati sensasi lezat m...,Makanan


1. CLEANSING

Tahapan proses cleansing data merupakan tahap pembersihan kata dari atribut yang tidak berpengaruh terhadap hasil klasifikasi sentimen. Komponen dokumen review memiliki beberapa atribut tidak berpengaruh terhadap sentimen diataranya url, html, emoji, simbol, angka dan tanda baca (~!@#$%^&*{}<>:|). Atribut yang tidak berpengaruh tersebut kemudian akan dihapus dan akan digantikan dengan karakter spasi.atribut-atribut tersebut dihapus dari teks untuk menghindari kebisingan (noise) dalam data. Setelah penghapusan, elemen-elemen ini digantikan dengan karakter spasi agar struktur kalimat tetap terjaga. Dengan pembersihan ini, data menjadi lebih fokus pada kata-kata atau frasa yang benar-benar relevan untuk menentukan sentimen, sehingga meningkatkan akurasi model dalam memprediksi sentimen.

In [ ]:
import re
import string
import nltk

# Fungsi ini bertujuan untuk menghapus URL dari teks.
def remove_url(ulasan):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', ulasan)

# Fungsi ini bertujuan untuk menghapus tag HTML dari teks.
def remove_html(ulasan):
  html = re.compile(r'<.#?>')
  return html.sub(r'', ulasan)

# Fungsi ini bertujuan untuk menghapus emoji dari teks.
def remove_emoji(ulasan):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', ulasan)

# Fungsi ini bertujuan untuk menghapus angka dari teks.
def remove_numbers(ulasan):
  ulasan = re.sub(r'\d+', '', ulasan)
  return ulasan


# Fungsi ini bertujuan untuk menghapus simbol dari teks, menyisakan hanya huruf, angka, dan spasi.
def remove_symbols(ulasan):
  ulasan = re.sub(r'[^a-zA-Z0-9\s]', '', ulasan)
  return ulasan

df['cleansing'] = df['isi'].apply(lambda x: remove_url(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_html(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_emoji(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_symbols(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_numbers(x))

df.head(5)

,judul,tanggal,isi,kategori,cleansing
0,Potret Cantik Enzy Storia Saat Kulineran di Pa...,"Rabu, 25 Sep 2024 22:00 WIB",Jakarta - Enzy Storia banjir pujian setelah ta...,Makanan,Jakarta Enzy Storia banjir pujian setelah tam...
1,Potret Mesra Sahrul Gunawan Saat Kulineran Bar...,"Rabu, 25 Sep 2024 21:00 WIB",Jakarta - Sahrul Gunawan dan istri gagal punya...,Makanan,Jakarta Sahrul Gunawan dan istri gagal punya ...
2,Ditegur Staf Restoran Gegara Bawa Makanan dari...,"Rabu, 25 Sep 2024 19:30 WIB",Jakarta - Seorang pengunjung restoran jadi sak...,Makanan,Jakarta Seorang pengunjung restoran jadi saks...
3,Unik! Burger Ini Isinya Kepiting Utuh Berukura...,"Rabu, 25 Sep 2024 19:00 WIB",Jakarta - Biasanya burger berisi patty atau da...,Makanan,Jakarta Biasanya burger berisi patty atau dag...
4,"Qobe Mie: Sensasi Mie Pedas Baru di Bogor, Lar...","Rabu, 25 Sep 2024 18:41 WIB",Jakarta - Ingin coba menikmati sensasi lezat m...,Makanan,Jakarta Ingin coba menikmati sensasi lezat mi...


2. CASE FOLDING

Pada tahap case folding huruf kapital pada semua dokumen ulasan diubah menjadi huruf kecil atau disebut lowercase. Hal ini bertujuan agar menghiangkan redudansi data yang hanya berbeda pada hurufnya saja. Misalnya, kata "Makanan" dan "makanan" dianggap sama dalam konteks analisis teks, tetapi tanpa case folding, keduanya akan dianggap berbeda oleh komputer. Dengan mengubah semua huruf menjadi kecil, kita menyatukan semua variasi penulisan menjadi satu bentuk yang konsisten, sehingga menghindari perhitungan yang berlebihan atau salah interpretasi data.

In [ ]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding
0,Potret Cantik Enzy Storia Saat Kulineran di Pa...,"Rabu, 25 Sep 2024 22:00 WIB",Jakarta - Enzy Storia banjir pujian setelah ta...,Makanan,Jakarta Enzy Storia banjir pujian setelah tam...,jakarta enzy storia banjir pujian setelah tam...
1,Potret Mesra Sahrul Gunawan Saat Kulineran Bar...,"Rabu, 25 Sep 2024 21:00 WIB",Jakarta - Sahrul Gunawan dan istri gagal punya...,Makanan,Jakarta Sahrul Gunawan dan istri gagal punya ...,jakarta sahrul gunawan dan istri gagal punya ...
2,Ditegur Staf Restoran Gegara Bawa Makanan dari...,"Rabu, 25 Sep 2024 19:30 WIB",Jakarta - Seorang pengunjung restoran jadi sak...,Makanan,Jakarta Seorang pengunjung restoran jadi saks...,jakarta seorang pengunjung restoran jadi saks...
3,Unik! Burger Ini Isinya Kepiting Utuh Berukura...,"Rabu, 25 Sep 2024 19:00 WIB",Jakarta - Biasanya burger berisi patty atau da...,Makanan,Jakarta Biasanya burger berisi patty atau dag...,jakarta biasanya burger berisi patty atau dag...
4,"Qobe Mie: Sensasi Mie Pedas Baru di Bogor, Lar...","Rabu, 25 Sep 2024 18:41 WIB",Jakarta - Ingin coba menikmati sensasi lezat m...,Makanan,Jakarta Ingin coba menikmati sensasi lezat mi...,jakarta ingin coba menikmati sensasi lezat mi...


3. TOKENIZATION

Tahap Tokenization merupakan pemotongan kata berdasarkan tiap kata yang menyusunnya menjadi potongan tunggal. Kata dalam dokumen yang dimaksud adalah kata yang dipisah oleh spasi, sehingga proses tokenisasi mengandalkan karakter spasi pada dokumen untuk melakukan pemisahan kata.
contoh, kalimat "Saya suka makan pizza" akan dipecah menjadi token ["Saya", "suka", "makan", "pizza"].

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize
0,Potret Cantik Enzy Storia Saat Kulineran di Pa...,"Rabu, 25 Sep 2024 22:00 WIB",Jakarta - Enzy Storia banjir pujian setelah ta...,Makanan,Jakarta Enzy Storia banjir pujian setelah tam...,jakarta enzy storia banjir pujian setelah tam...,"[jakarta, enzy, storia, banjir, pujian, setela..."
1,Potret Mesra Sahrul Gunawan Saat Kulineran Bar...,"Rabu, 25 Sep 2024 21:00 WIB",Jakarta - Sahrul Gunawan dan istri gagal punya...,Makanan,Jakarta Sahrul Gunawan dan istri gagal punya ...,jakarta sahrul gunawan dan istri gagal punya ...,"[jakarta, sahrul, gunawan, dan, istri, gagal, ..."
2,Ditegur Staf Restoran Gegara Bawa Makanan dari...,"Rabu, 25 Sep 2024 19:30 WIB",Jakarta - Seorang pengunjung restoran jadi sak...,Makanan,Jakarta Seorang pengunjung restoran jadi saks...,jakarta seorang pengunjung restoran jadi saks...,"[jakarta, seorang, pengunjung, restoran, jadi,..."
3,Unik! Burger Ini Isinya Kepiting Utuh Berukura...,"Rabu, 25 Sep 2024 19:00 WIB",Jakarta - Biasanya burger berisi patty atau da...,Makanan,Jakarta Biasanya burger berisi patty atau dag...,jakarta biasanya burger berisi patty atau dag...,"[jakarta, biasanya, burger, berisi, patty, ata..."
4,"Qobe Mie: Sensasi Mie Pedas Baru di Bogor, Lar...","Rabu, 25 Sep 2024 18:41 WIB",Jakarta - Ingin coba menikmati sensasi lezat m...,Makanan,Jakarta Ingin coba menikmati sensasi lezat mi...,jakarta ingin coba menikmati sensasi lezat mi...,"[jakarta, ingin, coba, menikmati, sensasi, lez..."


4. STOPWORD REMOVAL

Dalam tahapan proses Stopword Removal kata yang tidak memiliki pengaruh signifikan dalam kalimat akan dihilangkan. Dalam pre processing ini penulis menghapus stopword pada data ulasan berdasar daftar kalimat stopword diantaranya yaitu “yang”, “dan”, “di”, “dari”, dll.
contoh, kalimat "Saya pergi ke pasar dan membeli buah" akan diubah menjadi "Saya pergi pasar membeli buah" setelah proses stopword removal.

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize,stopword_removal
0,Potret Cantik Enzy Storia Saat Kulineran di Pa...,"Rabu, 25 Sep 2024 22:00 WIB",Jakarta - Enzy Storia banjir pujian setelah ta...,Makanan,Jakarta Enzy Storia banjir pujian setelah tam...,jakarta enzy storia banjir pujian setelah tam...,"[jakarta, enzy, storia, banjir, pujian, setela...",jakarta enzy storia banjir pujian tampil paris...
1,Potret Mesra Sahrul Gunawan Saat Kulineran Bar...,"Rabu, 25 Sep 2024 21:00 WIB",Jakarta - Sahrul Gunawan dan istri gagal punya...,Makanan,Jakarta Sahrul Gunawan dan istri gagal punya ...,jakarta sahrul gunawan dan istri gagal punya ...,"[jakarta, sahrul, gunawan, dan, istri, gagal, ...",jakarta sahrul gunawan istri gagal anak mencob...
2,Ditegur Staf Restoran Gegara Bawa Makanan dari...,"Rabu, 25 Sep 2024 19:30 WIB",Jakarta - Seorang pengunjung restoran jadi sak...,Makanan,Jakarta Seorang pengunjung restoran jadi saks...,jakarta seorang pengunjung restoran jadi saks...,"[jakarta, seorang, pengunjung, restoran, jadi,...",jakarta pengunjung restoran saksi mata keribut...
3,Unik! Burger Ini Isinya Kepiting Utuh Berukura...,"Rabu, 25 Sep 2024 19:00 WIB",Jakarta - Biasanya burger berisi patty atau da...,Makanan,Jakarta Biasanya burger berisi patty atau dag...,jakarta biasanya burger berisi patty atau dag...,"[jakarta, biasanya, burger, berisi, patty, ata...",jakarta burger berisi patty daging sapi bundar...
4,"Qobe Mie: Sensasi Mie Pedas Baru di Bogor, Lar...","Rabu, 25 Sep 2024 18:41 WIB",Jakarta - Ingin coba menikmati sensasi lezat m...,Makanan,Jakarta Ingin coba menikmati sensasi lezat mi...,jakarta ingin coba menikmati sensasi lezat mi...,"[jakarta, ingin, coba, menikmati, sensasi, lez...",jakarta coba menikmati sensasi lezat mie pedas...


In [ ]:
df.to_csv("/content/drive/My Drive/PPW/report/tugas_ppw/hasil_prepros.csv",encoding='utf8', index=False)

TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF adalah metode statistik yang digunakan untuk mengevaluasi pentingnya suatu kata dalam sebuah dokumen relatif terhadap koleksi dokumen lainnya. TF-IDF sering digunakan dalam tugas seperti penggalian teks, penambangan informasi, dan pemodelan pembelajaran mesin berbasis teks.
Term Frequency (TF), yang menghitung seberapa sering sebuah kata muncul dalam dokumen, dan Inverse Document Frequency (IDF), yang menilai seberapa jarang kata tersebut muncul di seluruh dokumen dalam koleksi.

TF-IDF bekerja dengan memberikan bobot lebih tinggi pada kata-kata yang sering muncul dalam sebuah dokumen, tetapi jarang muncul di dokumen lain, sehingga membantu mengidentifikasi kata-kata yang paling relevan. Kata-kata umum yang muncul di banyak dokumen, seperti "dan" atau "yang," akan mendapatkan bobot rendah karena dianggap kurang informatif.

In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/PPW/report/tugas_ppw/hasil_prepros.csv", sep=",")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

In [ ]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,aati,abad,abadi,abaikan,abang,abbey,abboud,abc,abdurrahman,abu,...,zaim,zaitun,zaman,zamannya,ziarah,zogba,zona,zone,zoo,zuleima
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.276395,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
